# PySpark DataFrames

A short introduction to PySpark is provided in this notebook. It is merely a starting point for exploring core features such as PySpark DataFrames.

In [ ]:
from datetime import datetime, date

import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import lit

## Start session

The main entry point for a PySpark application is provided by `SparkSession`. Before the unification that has been introduced in Spark 2.0, `SparkContext` used to be one of three different starting points. A session object can be initialized as it is shown in the following cell.

In [ ]:
# start session
spark = SparkSession.builder.getOrCreate()

## DataFrames

A PySpark `DataFrame` can be created in different ways, for example through a `pd.DataFrame` or a list of rows and and an explicit schema. Note that PySpark uses **lazy evaluation** for transformations on DataFrames, whereas pandas is based on **eager execution**.

In [ ]:
# create pandas DataFrame
pandas_df = pd.DataFrame(
    {
        'a': [1, 2, 3],
        'b': [2., 3., 4.],
        'c': ['string1', 'string2', 'string3'],
        'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
        'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
    }
)

# create DataFrame from pandas DataFrame
spark_df = spark.createDataFrame(pandas_df)

In [ ]:
# create spark DataFrame from list of rows
spark_df = spark.createDataFrame(
    [
        Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
        Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
        Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
    ]
)

In [ ]:
# create spark DataFrame with an explicit schema
spark_df = spark.createDataFrame(
    [
        (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
        (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
        (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
    ],
    schema='a long, b double, c string, d date, e timestamp'
)

In [ ]:
# show summary
spark_df.show()
spark_df.printSchema()

print(f'Number of rows: {spark_df.count()}')
print(f'Number of columns: {len(spark_df.columns)}')

## Accessing data

The following section demonstrates how data in a DataFrame can be accessed. Very often one needs to select certain columns or investigate the values contained in a specific row.

In [ ]:
# select columns (note that spark DataFrames are lazily evaluated)
one_col = spark_df.a

two_cols = spark_df.select('a', 'b')
two_cols = spark_df['a', 'b']

print(one_col)
print(two_cols)

In [ ]:
# return the first rows
list_of_first_rows = spark_df.head(2) # list_of_first_rows = spark_df.take(2)
list_of_last_rows = spark_df.tail(2)

print(list_of_first_rows)
print(list_of_last_rows)

In [ ]:
# collect distributed data to the driver (note that this may cause an out-of-memory error)
list_of_all_rows = spark_df.collect()

for row in list_of_all_rows:
    print(row)

In [ ]:
# get specific rows (with chosen columns)
one_row = spark_df.select('a', 'b').collect()[0] # note that this collects to the driver node
list_of_two_rows = spark_df.select('a', 'b').collect()[1:3]

print(one_row)
print(list_of_two_rows)

In [ ]:
# loop over columns
for cname in spark_df.columns:
    col = spark_df[cname]
    print(col)

# loop over rows (should be generally avoided for large datasets)
for row in spark_df.select('a', 'b').collect(): # collects to driver node
    print(row)

for idx, row in spark_df.select('a', 'b').toPandas().iterrows(): # collects to driver node
    print(row)

for row in spark_df.select('a', 'b').toLocalIterator(): # iterates over rows
    print(row)

spark_df.select('a', 'b').rdd.foreach(lambda row: print(row)) # applies to each row

In [ ]:
# filter rows of DataFrame
new_df = spark_df.filter(spark_df.a == 1) # based on condition
new_df = spark_df.filter('a == 1') # based on SQL expression

new_df.show()

## Manipulating data

Operations on DataFrames are either **transformations** (producing new DataFrames) or **actions** (triggering a computation in order to return a value). Examples of actions are `collect()`, `count()`, `show()` and `reduce()`. Transformations are for instance `filter()` and `map()`.




In [ ]:
# create DataFrame
df = spark.createDataFrame(
    [
        ['red', 'banana', 1, 10],
        ['blue', 'banana', 2, 20],
        ['red', 'carrot', 3, 30],
        ['blue', 'grape', 4, 40],
        ['red', 'carrot', 5, 50],
        ['black', 'carrot', 6, 60],
        ['red', 'banana', 7, 70],
        ['red', 'grape', 8, 80]
    ],
    schema=['color', 'fruit', 'v1', 'v2']
)

df.show()
df.printSchema()

In [ ]:
# group by color and average
color_avgs = df.groupby('color').avg()

color_avgs.show()

In [ ]:
# rename column
new_df = df.withColumnRenamed('v1', 'v3')
new_df = new_df.withColumnRenamed('v2', 'v4')

new_df.show()

In [ ]:
# drop column
new_df = df.drop('v2')

new_df.show()

In [ ]:
# add columns (with constant values)
new_df = df.withColumn('new_col1', lit('new_val'))
new_df = new_df.withColumn('new_col2', lit(0))

new_df.show()
new_df.printSchema()

In [ ]:
# add column (from existing one)
new_df = df.withColumn('v3', df.v2 * 2)

new_df.show()

In [ ]:
# filter rows of DataFrame
new_df = df.filter(df.fruit == 'banana') # based on condition

new_df.show()

In [ ]:
# apply row-wise function
new_rdd = df.rdd.map(lambda row: (row.color, row.fruit, row.v1 * 2))
new_df = new_rdd.toDF(('color', 'fruit', 'double_v1'))

new_df.show()

## SQL queries

DataFrames can also be accessed through SQL-like queries. Some simple examples of such queries are shown in the cells below.

In [ ]:
# register DataFrame as SQL table
df.createOrReplaceTempView('table')

In [ ]:
# run SQL-style query
spark.sql(
    'SELECT * FROM table WHERE v2 >= 30'
).show()

In [ ]:
# run SQL-style query
spark.sql(
    '''SELECT color, fruit, v2 FROM table
    WHERE fruit in ('banana', 'grape')'''
).show()

In [ ]:
# run SQL-style query
spark.sql(
    '''SELECT color, fruit, v1 FROM table
    ORDER BY fruit'''
).show()

In [ ]:
# run SQL-style query
spark.sql(
    '''SELECT fruit, COUNT(*) AS count FROM table
    GROUP BY fruit'''
).show()

In [ ]:
# run SQL-style query
spark.sql(
    'SELECT COUNT(*) AS count FROM table'
).show()

## Stop session

In [ ]:
# stop session
spark.stop()